# Inputs

Generate json input files to run an experiment with `doframework`. 

The products of this script will be written to `inputs` bucket as specified in `configs.yaml`. 

In [1]:
import json
import yaml
import os

import numpy as np
from doframework.core.inputs import generate_id
from doframework import api

# Configs

In [2]:
configs_path = '..' # TODO: provide path
configs_file = 'configs.yaml'

with open(os.path.join(configs_path,configs_file),'r') as file:
    try:
        configs = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)        

# S3

In [3]:
buckets = api._get_buckets(configs)

# File Format

In [4]:
output_prefix = 'input'
output_suffix = 'json'

# Input Combinations

In [5]:
L = 10.0 # length scale 
dimensions = [2,3]
interval = [0.0,L]

In [6]:
vertex_ranges = [d*[interval] for d in dimensions]
vertex_nums = [[(d+1)+int(j*np.log(L)) for j in range(1,d+1)] for d in dimensions] # TODO: change
value_ranges = [[[np.min(np.array(r))-(np.max(np.array(r))-np.min(np.array(r)))/2,
                np.max(np.array(r))-(np.max(np.array(r))-np.min(np.array(r)))/2]]+
                [[np.min(np.array(r))-(2*np.max(np.array(r))-np.min(np.array(r)))/2,
                2*np.max(np.array(r))-(2*np.max(np.array(r))-np.min(np.array(r)))/2]] for r in vertex_ranges]
policies = [[2+j for j in range(1,int(max(N)/3),3)] for N in vertex_nums]

omega_ratios = [0.7,0.9]
omega_scales = [0.001,0.01]
Ns = [500,1000]
noises = [0.5,0.1]
data_scales = [0.6,0.8]

### produce output

In [7]:
limit = None # TODO: test with limit = 1
outputs = []

In [8]:
for d,vertex_range in enumerate(vertex_ranges):
    for vertex_num in vertex_nums[d][:limit]:
        for value_range in value_ranges[d][:limit]:
            for ratio in omega_ratios[:][:limit]:
                for omega_scale in omega_scales[:limit]:
                    for N in Ns[:limit]:
                        for noise in noises[:limit]:
                            for policy in policies[d][:limit]:
                                for data_scale in data_scales[:limit]:

                                    output = {}
                                    output['f'] = {}
                                    output['f']['vertices'] = {}
                                    output['f']['values'] = {}
                                    output['omega'] = {}
                                    output['data'] = {}
                                    
                                    output['f']['vertices']['num'] = vertex_num
                                    output['f']['vertices']['range'] = vertex_range
                                    output['f']['values']['range'] = value_range
                                    output['omega']['ratio'] = ratio
                                    output['omega']['scale'] = omega_scale
                                    output['data']['N'] = N
                                    output['data']['noise'] = noise
                                    output['data']['policy_num'] = policy
                                    output['data']['scale'] = data_scale
                                    
                                    output_id = generate_id()
                                    file = ''.join(['_'.join([output_prefix,output_id]),'.',output_suffix])
                                    output['input_file_name'] = file                             
                                    
                                    outputs.append(output)

                                    api._get_s3_object(configs).Bucket(buckets['inputs']).\
                                        put_object(Body=json.dumps(output),Key=file)

print('Uploaded {} input files to bucket {}.'.format(len(outputs),buckets['inputs']))

Uploaded 320 input files to bucket autodo-simulations-inputs.
